In [ ]:
import boto3

In [ ]:
s3 = boto3.resource('s3',
    aws_access_key_id='',
    aws_secret_access_key=''
)

In [ ]:
try:
    s3.create_bucket(Bucket='cs1660-hw3-bucket', CreateBucketConfiguration={
        'LocationConstraint': 'us-west-2'})
except Exception as e:
    print(e)

In [ ]:
bucket = s3.Bucket('cs1660-hw3-bucket')

In [ ]:
bucket.Acl().put(ACL='public-read')

In [ ]:
dyndb = boto3.resource('dynamodb',
    region_name='us-west-2',
    aws_access_key_id='',
    aws_secret_access_key=''
)

In [ ]:
try:
    table = dyndb.create_table(
        TableName='DataTable',
        KeySchema=[
            {
                'AttributeName': 'PartitionKey',
                'KeyType': 'HASH'
            },
            {
                'AttributeName': 'RowKey',
                'KeyType': 'RANGE'
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'PartitionKey',
                'AttributeType': 'S'
            },
            {
                'AttributeName': 'RowKey',
                'AttributeType': 'S'
            },
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 5,
            'WriteCapacityUnits': 5
        }
)
except Exception as e:
    print(e)
    table = dyndb.Table("DataTable")

In [ ]:
table.meta.client.get_waiter('table_exists').wait(TableName='DataTable')

In [ ]:
import csv

In [ ]:
with open('experiments.csv', 'r') as csvfile:
    csvf = csv.reader(csvfile, delimiter=',', quotechar='|')
    next(csvf)# Skips the header row in the csv file
    for item in csvf:
        print(item)
        body = open('datafiles/'+item[5], 'rb')
        s3.Object('cs1660-hw3-bucket', item[5]).put(Body=body )
        md = s3.Object('cs1660-hw3-bucket', item[5]).Acl().put(ACL='public-read')

        url = "https://s3-us-west-2.amazonaws.com/cs1660-hw3-bucket/"+item[5]
        metadata_item = {'PartitionKey': item[0], 'RowKey': item[1],
            'temp': item[2], 'conductivity': item[3], 'concentration': item[4], 'url': url}
        try:
            table.put_item(Item=metadata_item)
        except:
            print("item may already be there or another failure")

In [ ]:
response = table.get_item(
    Key={
        'PartitionKey': 'experiment1',
        'RowKey': '1'
    }
)
item = response['Item']

In [ ]:
print(item)